In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.tensorboard import SummaryWriter

In [2]:
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

In [3]:
data = pd.read_csv('./data/GDP.csv')
data.set_index('date',inplace=True)
base = data.loc[2000]
scaled_data = data/base
years = data.index
years = list(map(int,years))
inputs = torch.tensor(scaled_data.iloc[:-1].values, dtype=torch.float32)
labels = torch.tensor(scaled_data.iloc[1:].values, dtype=torch.float32)
train_sequence = sum([i>=1970 and i<=2000 for i in years])
test_sequence = sum([i>2000 for i in years])
train_data = inputs[:train_sequence]
train_label = labels[:train_sequence]
test_data = inputs[train_sequence:]
test_label = labels[train_sequence:]

In [4]:
class Net(nn.Module):
    def __init__(self,input_size,hidden_size):
        super(Net,self).__init__()
        self.rnn = nn.LSTM(input_size,hidden_size)
        self.fc = nn.Linear(hidden_size,1)
    def forward(self,X):
        X = X[:,:,None]
        X, _ = self.rnn(X)
        X = self.fc(X)
        X = X[:,:,0]
        return X

In [5]:
model = Net(input_size=1,hidden_size=5)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
model.to(device)
criterion.to(device)

MSELoss()

In [6]:
steps = 50000
#writer = SummaryWriter(log_dir='./log')
#scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=1000,gamma=0.1)
for step in range(steps):
    # train
    train_data, train_label = train_data.to(device), train_label.to(device)
    optimizer.zero_grad()
    train_output = model(train_data)
    train_loss = criterion(train_output,train_label)
    train_loss.backward()
    optimizer.step()
    for para_group in optimizer.param_groups:
        current_lr = para_group['lr']
    if (step+1)%1000==0:
        print(f'{step+1}/{steps} lr={current_lr} train_loss={train_loss.item()}')
        #writer.add_scalar('Loss/train',train_loss.item(),step+1)
    #scheduler.step()
    
    # eval
    if (step+1)%10000==0:
        model.eval()
        with torch.no_grad():
            test_data, test_label = test_data.to(device), test_label.to(device)
            test_output = model(test_data)
            test_loss = criterion(test_output,test_label)
            print(f'{step+1}/{steps} test_loss={test_loss.item()}')
            #writer.add_scalar('Loss/test',test_loss.item(),step+1)

1000/50000 lr=0.001 train_loss=0.007370236795395613
2000/50000 lr=0.001 train_loss=0.002797957742586732
3000/50000 lr=0.001 train_loss=0.0020442563109099865
4000/50000 lr=0.001 train_loss=0.0015029150526970625
5000/50000 lr=0.001 train_loss=0.0011940557742491364
6000/50000 lr=0.001 train_loss=0.0008794350433163345
7000/50000 lr=0.001 train_loss=0.0006661438383162022
8000/50000 lr=0.001 train_loss=0.0006025622133165598
9000/50000 lr=0.001 train_loss=0.0005734245642088354
10000/50000 lr=0.001 train_loss=0.0005474589415825903
10000/50000 test_loss=0.12449415773153305
11000/50000 lr=0.001 train_loss=0.0005186108755879104
12000/50000 lr=0.001 train_loss=0.0005030724569223821
13000/50000 lr=0.001 train_loss=0.0004820883914362639
14000/50000 lr=0.001 train_loss=0.00046593527076765895
15000/50000 lr=0.001 train_loss=0.0004541445814538747
16000/50000 lr=0.001 train_loss=0.0004427362291608006
17000/50000 lr=0.001 train_loss=0.0004451455897651613
18000/50000 lr=0.001 train_loss=0.0004419206816237